In [ ]:
!pip install opencv-python

In [ ]:
!pip install mediapipe
!wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 33.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled p

In [ ]:
!pip install openface-test

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 MB 51.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of scikit-image to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.8/251.8 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.1/346.1 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.5/313.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━

In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import time
import numpy as np
import pandas as pd
import os
from openface.multitask_model import MultitaskPredictor
from openface.face_detection import FaceDetector
import torch

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def load_videos(person: str, emotion: str, iter: int) -> cv2.VideoCapture | None:
    base_dir = "/content/drive/MyDrive/video_data/"
    final_filename = f"{person}_{emotion}_{iter}.mov"
    final_path = os.path.join(base_dir, final_filename)

    cap = cv2.VideoCapture(final_path)

    if not cap.isOpened():
        print(f"Error: Could not open video file at {final_path}")
        return None

    print(f"Successfully loaded video: {final_path}")
    return cap

video_capture = load_videos("keshia", emotion="happy", iter=1)

if video_capture:
    video_capture.release()


Successfully loaded video: /content/drive/MyDrive/video_data/keshia_happy_1.mov


In [ ]:
import math
def get_euler_angles(matrix: np.ndarray) -> dict:
    if matrix is None or matrix.shape != (4, 4):
        return {'yaw': np.nan, 'pitch': np.nan, 'roll': np.nan}

    R = matrix[:3, :3]
    pitch = -np.arcsin(R[2, 0])

    yaw = np.arctan2(R[1, 0], R[0, 0])

    roll = np.arctan2(R[2, 1], R[2, 2])

    yaw_deg = math.degrees(yaw)
    pitch_deg = math.degrees(pitch)
    roll_deg = math.degrees(roll)

    return {'yaw': yaw_deg, 'pitch': pitch_deg, 'roll': roll_deg}


In [ ]:
face_model_path = '/content/drive/MyDrive/Alignment_RetinaFace.pth'
multitask_model_path = '/content/drive/MyDrive/MTL_backbone.pth'
!mkdir -p weights
!ln -s "/content/drive/MyDrive/mobilenetV1X0.25_pretrain.tar" "./weights/mobilenetV1X0.25_pretrain.tar"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def video_processing(file_path, person, emotion, iteration):
  final_output = pd.DataFrame()
  try:
      multitask_model = MultitaskPredictor(model_path=multitask_model_path, device=device)
      face_detector = FaceDetector(model_path=face_model_path, device='cuda')
  except NameError:
      print("Error: MultitaskPredictor class not found. Ensure the custom library is imported.")
      exit()

  try:
      base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
      options = vision.FaceLandmarkerOptions(base_options=base_options,
                                            output_face_blendshapes=True,
                                            output_facial_transformation_matrixes=True,
                                            num_faces=1)
      detector = vision.FaceLandmarker.create_from_options(options)
  except Exception as e:
      print(f"Error initializing MediaPipe FaceLandmarker: {e}")
      return final_output

  cap = load_videos(person, emotion, iteration)

  if not cap.isOpened():
      print(f"Error: Could not open video file at {file_path}")
      exit()

  print(f"Starting analysis on video: {file_path}")
  frame_count = 0

  while cap.isOpened():
      ret, frame = cap.read()
      if not ret:
          break

      if not cv2.imwrite('/tmp/temp_frame.jpg', frame):
          print(f"Error: Could not write frame {frame_count} to temporary file.")
          continue

      try:
          cropped_face, dets = face_detector.get_face('/tmp/temp_frame.jpg')
      except NameError:
          print("Error: 'face_detector' object not defined.")
          cap.release()
          exit()

      detection_result = None
      blendshape_data = {}
      pose_angles = {'yaw': np.nan, 'pitch': np.nan, 'roll': np.nan}

      try:
          rgb_full_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
          mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_full_frame)
          detection_result = detector.detect(mp_image)

          if detection_result.face_blendshapes and detection_result.face_blendshapes[0]:
              for category in detection_result.face_blendshapes[0]:
                  blendshape_data[f'bs_{category.category_name}'] = category.score

          if detection_result.facial_transformation_matrixes:
              matrix = detection_result.facial_transformation_matrixes[0]
              pose_angles = get_euler_angles(matrix)
      except Exception as e:
          print(f"Frame {frame_count}: MediaPipe detection failed: {e}")

      if cropped_face is not None and dets is not None:

          with torch.no_grad():
            emotion_logits, gaze_output, au_output = multitask_model.predict(cropped_face)
          emotion_index = torch.argmax(emotion_logits, dim=1).item()
          gaze_data = gaze_output.cpu().squeeze().numpy()
          au_data = au_output.cpu().squeeze().numpy()

          # print(f"Frame {frame_count}: Emotion Index: {emotion_index}, Gaze: {gaze_data}, AU shape: {au_output.shape}")
          new_row_data = {
              'frame_count': frame_count,
              'emotion_index': emotion_index,
              'gaze_yaw': gaze_data[0] if gaze_data.size > 0 else np.nan,
              'gaze_pitch': gaze_data[1] if gaze_data.size > 1 else np.nan,
              'au_shape_dim1': au_data[0] if au_data.size > 0 else np.nan,
              'au_shape_dim2': au_data[1] if au_data.size > 1 else np.nan,
              'au_shape_dim3': au_data[2] if au_data.size > 1 else np.nan,
              'au_shape_dim4': au_data[3] if au_data.size > 1 else np.nan,
              'au_shape_dim5': au_data[4] if au_data.size > 1 else np.nan,
              'au_shape_dim6': au_data[5] if au_data.size > 1 else np.nan,
              'au_shape_dim7': au_data[6] if au_data.size > 1 else np.nan,
              'au_shape_dim8': au_data[7] if au_data.size > 1 else np.nan,

              'mp_pose_yaw_deg': pose_angles['yaw'],
              'mp_pose_pitch_deg': pose_angles['pitch'],
              'mp_pose_roll_deg': pose_angles['roll'],
              'detection_result': 'Extracted Data' if blendshape_data or pose_angles['yaw'] is not np.nan else 'No MP Data'
          }
          new_row_data.update(blendshape_data)
          new_row_df = pd.DataFrame(new_row_data, index=[0])
          final_output = pd.concat([final_output, new_row_df], ignore_index=True)
      else:
          print(f"Frame {frame_count}: No face detected.")

      frame_count += 1

  cap.release()
  cv2.destroyAllWindows()
  print("Video processing complete.")
  return final_output

In [ ]:
# people = ['keshia', 'fai', 'emma', 'grant]
people = ['grant']
emotions = ['angry', 'happy', 'sad', 'fear', 'disgust', 'neutral', 'surprise']
# emotions = ['surprise']
!mkdir -p processed_videos

for person in people:
  for emotion in emotions:
    for i in range(1,11):
      file_path = f"/content/drive/MyDrive/video_data/{person}_{emotion}_{i}.mov"
      processed_video = video_processing(file_path, person, emotion, i)
      processed_video.reset_index(inplace = True, drop = True)
      processed_video.to_csv(f"processed_videos/{person}_{emotion}_{i}_processed.csv", index = 'ignore')

/usr/local/lib/python3.12/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name tf_efficientnet_b0_ns to current tf_efficientnet_b0.ns_jft_in1k.
  model = create_fn(


Loading multitask model from /content/drive/MyDrive/MTL_backbone.pth...
Loading pretrained model from /content/drive/MyDrive/Alignment_RetinaFace.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:300
Successfully loaded video: /content/drive/MyDrive/video_data/grant_angry_1.mov
Starting analysis on video: /content/drive/MyDrive/video_data/grant_angry_1.mov
Video processing complete.


/usr/local/lib/python3.12/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name tf_efficientnet_b0_ns to current tf_efficientnet_b0.ns_jft_in1k.
  model = create_fn(


Loading multitask model from /content/drive/MyDrive/MTL_backbone.pth...
Loading pretrained model from /content/drive/MyDrive/Alignment_RetinaFace.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:300
Successfully loaded video: /content/drive/MyDrive/video_data/grant_angry_2.mov
Starting analysis on video: /content/drive/MyDrive/video_data/grant_angry_2.mov
Video processing complete.


/usr/local/lib/python3.12/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name tf_efficientnet_b0_ns to current tf_efficientnet_b0.ns_jft_in1k.
  model = create_fn(


...

Loading multitask model from /content/drive/MyDrive/MTL_backbone.pth...
Loading pretrained model from /content/drive/MyDrive/Alignment_RetinaFace.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:300
Successfully loaded video: /content/drive/MyDrive/video_data/grant_surprise_8.mov
Starting analysis on video: /content/drive/MyDrive/video_data/grant_surprise_8.mov
Video processing complete.


/usr/local/lib/python3.12/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name tf_efficientnet_b0_ns to current tf_efficientnet_b0.ns_jft_in1k.
  model = create_fn(


Loading multitask model from /content/drive/MyDrive/MTL_backbone.pth...
Loading pretrained model from /content/drive/MyDrive/Alignment_RetinaFace.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:300
Successfully loaded video: /content/drive/MyDrive/video_data/grant_surprise_9.mov
Starting analysis on video: /content/drive/MyDrive/video_data/grant_surprise_9.mov
Video processing complete.


/usr/local/lib/python3.12/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name tf_efficientnet_b0_ns to current tf_efficientnet_b0.ns_jft_in1k.
  model = create_fn(


Loading multitask model from /content/drive/MyDrive/MTL_backbone.pth...
Loading pretrained model from /content/drive/MyDrive/Alignment_RetinaFace.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:300
Successfully loaded video: /content/drive/MyDrive/video_data/grant_surprise_10.mov
Starting analysis on video: /content/drive/MyDrive/video_data/grant_surprise_10.mov
Video processing complete.


In [ ]:
import shutil
from google.colab import files
shutil.make_archive('processed_videos_grant', 'zip', 'processed_videos_grant')
# files.download('processed_videos.zip')
files.download('processed_videos_grant.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>